In [ ]:
import torch
import plyfile
import numpy as np
import open3d as o3d

import matplotlib
import matplotlib.pyplot as plt

import os
import sys
sys.path.append(os.path.abspath(".."))
from lib.utils.image_utils import color_mapping

def RGB2SH(rgb):
    C0 = 0.28209479177387814
    return (rgb - 0.5) / C0

def save_2dgs_as_ply(path, position, scale, rotation, opacity, features_dc):
    # 将输入转换为numpy数组
    position = position.detach().cpu().numpy()
    scale = scale.detach().cpu().numpy()
    rotation = torch.nn.functional.normalize(rotation.detach()).cpu().numpy()
    opacity = opacity.detach().cpu().numpy()
    if type(features_dc) is torch.nn.parameter.Parameter:
        features_dc = features_dc.detach().squeeze()
    features_dc = features_dc.cpu().numpy()

    # 创建顶点数组
    vertices = np.zeros(position.shape[0], dtype=[
        ('x', 'f4'), ('y', 'f4'), ('z', 'f4'),
        ('nx', 'f4'), ('ny', 'f4'), ('nz', 'f4'),
        ('f_dc_0', 'f4'), ('f_dc_1', 'f4'), ('f_dc_2', 'f4'),
        ('opacity', 'f4'),
        ('scale_0', 'f4'), ('scale_1', 'f4'), ('scale_2', 'f4'),
        ('rot_0', 'f4'), ('rot_1', 'f4'), ('rot_2', 'f4'), ('rot_3', 'f4')
    ])

    position -= np.mean(position, axis=0)

    # 填充顶点数据
    vertices['x'] = position[:, 0]
    vertices['y'] = position[:, 1]
    vertices['z'] = position[:, 2]
    vertices['f_dc_0'] = features_dc[:, 0]
    vertices['f_dc_1'] = features_dc[:, 1]
    vertices['f_dc_2'] = features_dc[:, 2]
    vertices['opacity'] = opacity[:, 0]
    vertices['scale_0'] = scale[:, 0]
    vertices['scale_1'] = scale[:, 1]
    vertices['scale_2'] = np.ones_like(scale[:, 0]) * -100.
    vertices['rot_0'] = rotation[:, 0]
    vertices['rot_1'] = rotation[:, 1]
    vertices['rot_2'] = rotation[:, 2]
    vertices['rot_3'] = rotation[:, 3]

    # vertices = vertices[ids]
    print(f"num vertices: {len(vertices)}")

    # 创建PLY文件
    el = plyfile.PlyElement.describe(vertices, 'vertex')
    plyfile.PlyData([el]).write(path)


def plt_hist(data):
    plt.figure(figsize=(10, 6))
    plt.hist(data, bins=50, alpha=0.75)
    plt.title('Scale Norm Distribution')
    plt.xlabel('Scale Norm')
    plt.ylabel('Frequency')
    plt.grid(True, alpha=0.3)
    plt.show()


In [ ]:
model_path = "../output/default/test/scene_ks3/models/model_it_35000.pth"
model_params, first_iter = torch.load(model_path)
(active_sh_degree, _xyz, _features_dc, 
 _features_rest, _scaling, _rotation, 
 _opacity, max_radii2D, xyz_gradient_accum, 
 denom, opt_dict, spatial_lr_scale)= model_params[0]
print(f"num gaussians: {_xyz.shape[0]}")

scale = torch.exp(_scaling.detach())

# scale_norm = torch.log10(torch.norm(scale, dim=1)).cpu().numpy()
# plt_hist(scale_norm)

# area = torch.log10(scale[:, 0] * scale[:, 1]).cpu().numpy()
# plt_hist(area)

# opa = torch.sigmoid(_opacity.detach()).cpu().numpy()
# plt_hist(opa)


In [ ]:
points = _xyz.detach()
z = points[:, 2]
max_q = torch.quantile(z, 0.9)
min_q = torch.quantile(z, 0.1)
z = (z - min_q) / (max_q - min_q)
z = z.clamp(0, 1)
colormap = matplotlib.colormaps["rainbow"]
color = color_mapping(z, colormap)
f_dc = RGB2SH(color)

save_2dgs_as_ply("2dgs.ply", 
                 position=_xyz, 
                 scale=_scaling, 
                 rotation=_rotation, 
                 opacity=_opacity, 
                 features_dc=f_dc)


In [ ]:
# 保存成 mesh ply

xyz = _xyz.detach().cpu().numpy()

z = _xyz[:, 2].detach().cpu()
max_q = torch.quantile(z, 0.9)
min_q = torch.quantile(z, 0.1)
z = (z - min_q) / (max_q - min_q)
z = z.clamp(0, 1)

colormap=matplotlib.colormaps["rainbow"]
color = color_mapping(z, colormap)

# save as ply
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)
pcd.colors = o3d.utility.Vector3dVector(color.cpu().numpy().astype(np.float64))

o3d.io.write_point_cloud(f"xyz_it_{first_iter}.ply", pcd)
